# RAG: Simple generation

In [./rag_0_indexing.ipynb](./rag_0_indexing.ipynb), we indexed a geography text book.

In this notebook, we ask questions and the RAG system finds us answers.

## Set up.

Install the necessary packages, set up the API keys etc.

In [1]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain import hub

from dotenv import load_dotenv
load_dotenv("../keys.env");

In [2]:
PROVIDER = "Google"
#PROVIDER = "OpenAI"
PERSIST_DIR = "vectordb"

In [3]:
if PROVIDER == "Google":
    from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
else:
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    embeddings = OpenAIEmbeddings()
    model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

## Load the vector db from disk and create a chain to ask questions

The question is embedded with the same embedding function as the paragraphs.
Similar chunks are found and added to the context.
The LLM uses this information to answer the question

In [4]:
vectorstore = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [5]:
retrieval_chain = RunnablePassthrough() | retriever
retrieval_chain.invoke("What types of rock do you find in the Upper Lias?")

[Document(metadata={'paragraph': 1104, 'source': 'geography'}, page_content='The Lias has been divided in England into three groups, the Upper, Middle, and Lower. The Upper Lias consists first of sands, which were formerly regarded as the base of the Oolite, but which, according to Dr. Wright, are by their fossils more properly referable to the Lias; secondly, of clay shale and thin beds of limestone. The Middle Lias, or marl-stone series, has been divided into three zones; and the Lower Lias, according to the labours of Quenstedt, Oppel, Strickland, Wright, and others, into seven zones, each marked by its own group of fossils. This Lower Lias averages from 600 to 900 feet in thickness.'),
 Document(metadata={'paragraph': 1103, 'source': 'geography'}, page_content='Lias.—The English provincial name of Lias has been very generally adopted for a formation of argillaceous limestone, marl, and clay, which forms the base of the Oolite, and is classed by many geologists as part of that group

In [6]:
def add_docs_to_context(docs):
    return "\n".join(doc.page_content for doc in docs)

In [7]:
rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [8]:
rag_chain = (
    {"context": retriever | add_docs_to_context, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [9]:
rag_chain.invoke("What types of rock do you find in the Upper Lias?")

'The Upper Lias consists of sands, clay shale, and thin beds of limestone. These rocks were formerly considered part of the Oolite, but are now classified as Lias due to their fossil content. The Upper Lias is characterized by a distinct set of fossils, with only 37 out of 256 mollusk species found in the Upper Lias also present in the Inferior Oolite. \n'

## Next step

In [./rag_2_bettergen.ipynb](rag_2_bettergen.ipynb), we do a few things that make the RAG generation better.